In [1]:
#%%
import sys
# sys.path.append("/home/baokhanh/")
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

/home/linhnguyen/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)
/home/linhnguyen/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack.errors package is deprecated. Please use slack_sdk.errors package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


In [6]:

drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

sa_ana = Path('/home/linhnguyen/.config/ts-dp-v2.json')
adc = Path('/home/linhnguyen/.config/gcloud/application_default_credentials.json')
bq_client = BigqueryClient(billing_project_id='prj-ts-p-analytic-8057', adc_file_path= adc) #sa_file_path = sa_ana)
bq_repo = BigqueryService(bq_client)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

linh_id = "U047QMYB9TQ"
channel_id = "C04MYU3L7LN"
bot_id = "U0480LX5468"

In [3]:
def dayrange_recon(day_from_curr: int = 25) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, start_date, end_date

recon_month, start_date, end_date = dayrange_recon()

In [4]:
recon_month

'2023-08'

In [5]:
# create directory
path = '/home/linhnguyen/04.Reconcile/06.TPB/drop_off/'

import os

if not os.path.exists(f'{path}{recon_month}'):
    os.makedirs(f'{path}{recon_month}')

# Query card list

In [7]:
query = """
with dropoff_list as (
  select  distinct 
          application_id as appid
          , 'dropoff' as status
          , s.created_at as dropoff_at
  from `prj-ts-p-lakehouse-8435.dop.sent_applications` s
  where s.created_at >= '2022-10-01'
    and type = 'push_app'
    and s.status = 'success'
    and transaction_id not like 'success:%'
    and transaction_id not like 'failure:%'
)

, smv as (
  select 
  smv.*
  , DENSE_RANK() OVER (PARTITION BY lead_unique_token ORDER BY smv_issued_at ASC) AS rank
  from `prj-ts-p-lakehouse-8435.tpb.tpb_smv_in_dop_map` smv
)

SELECT DISTINCT 
    dop.lead_unique_token
    , dop.appid
    , cif
    , card_no
    , ten_kh
    , smv.smv_card_category
    , smv_issued_at
    , smv_active_at
    , FORMAT_DATE('%Y-%m', DATE(dropoff_list.dropoff_at)) as dropoff_month
    , FORMAT_DATE('%Y-%m', DATE(dop.first_card_issued_at)) as smv_issue_month
    , FORMAT_DATE('%Y-%m', DATE(dop.first_card_active_at)) as smv_active_month
    , case when dop.mkt_channel like '%SMS%' then dop.mkt_channel ELSE 'EVO_DIGITAL' END AS mkt_channel
    , dop.has_card_issued
    , dop.has_card_active
    , CASE WHEN dropoff_list.status = 'dropoff' then 1 else 0 end as has_dropoff
FROM `prj-ts-p-lakehouse-8435.tpb.tpb_dop_full_map` dop
JOIN (select * from smv where rank=1) smv
    ON dop.lead_unique_token = smv.lead_unique_token
LEFT JOIN dropoff_list 
    ON dop.appid = dropoff_list.appid

WHERE 1=1
    AND has_card_issued = 1
    AND FORMAT_DATE('%Y-%m', DATE(dop.first_card_issued_at)) = FORMAT_DATE('%Y-%m',DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
"""

query2 = "select * from `prj-ts-p-analytic-8057.da_quynh.linh4` where dropoff_month is not null"

df = bq_repo.get_data_from_query_into_pandas(query_str=query)

print("Done loading data on " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
display(df.head())


Done loading data on 2023-09-07 15:30:01


,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,dropoff_month,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff
0,UiK-GuGH,6401560,01580490,498796xxxxxx8945,TRUONG HOANG HUY,VISA CREDIT GOLD EVO CARD (9024),2023-08-11 16:13:39.000057,2023-08-11,None,2023-08,2023-08,EVO_DIGITAL,1,1,0
1,YRFfBE3y,6326718,04028607,498796xxxxxx3049,NGUYEN MINH THANH,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 11:45:56.999952,2023-08-08,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
2,QiK97rHg,6220574,02009151,498796xxxxxx2147,NGUYEN THI MY LINH,VISA CREDIT GOLD EVO CARD (9024),2023-08-05 09:00:08.000035,NaT,2023-07,2023-08,None,SMS-MBF WITH OFFER,1,0,1
3,cRFfBLtq,6330221,11830786,498796xxxxxx9377,NGUYEN ANH HAO,VISA EVO SILVER CARD (9023),2023-08-03 17:47:18.999945,2023-08-07,None,2023-08,2023-08,EVO_DIGITAL,1,1,0
4,eRFfCkzv,6375837,11888418,498796xxxxxx5729,NGUYEN THI NHUNG,VISA EVO SILVER CARD (9023),2023-08-09 15:57:52.999977,NaT,2023-08,2023-08,None,EVO_DIGITAL,1,0,1


In [8]:
df[df.duplicated(['lead_unique_token'], keep=False)].sort_values(by='lead_unique_token')

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,dropoff_month,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff
448,QiK-AUfo,6296702,11794976,498796xxxxxx1999,TRUONG VAN THONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-09 13:52:42.000009,2023-08-09,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1617,QiK-AUfo,6296702,11794976,498796xxxxxx1999,TRUONG VAN THONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-09 13:52:42.000009,2023-08-09,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
1582,QiK948pL,6175908,09499159,498796xxxxxx6554,BUI TIEN THANH,VISA EVO SILVER CARD (9023),2023-08-03 18:01:11.000064,2023-08-04,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
1675,QiK948pL,6175908,09499159,498796xxxxxx6554,BUI TIEN THANH,VISA EVO SILVER CARD (9023),2023-08-03 18:01:11.000064,2023-08-04,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1858,QiK96CmB,6193816,11813079,498796xxxxxx5199,NGUYEN HOANG TAM,VISA EVO SILVER CARD (9023),2023-08-01 20:21:56.000016,2023-08-02,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,kRXv1qo-,6270292,03221257,498796xxxxxx0941,LE THI THANH TUYEN,VISA EVO SILVER CARD (9023),2023-08-02 00:36:00.000000,2023-08-09,2023-08,2023-08,2023-08,SMS-MBF WITH OFFER,1,1,1
2215,kRXvkY4w,6234908,02021847,498796xxxxxx1558,DUONG NGOC THU,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 13:48:41.999990,2023-08-07,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
245,kRXvkY4w,6234908,02021847,498796xxxxxx1558,DUONG NGOC THU,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 13:48:41.999990,2023-08-07,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1833,kRXwFTuc,6302327,10145525,498796xxxxxx4100,NGUYEN VAN PHI,VISA EVO SILVER CARD (9023),2023-08-01 19:24:24.999984,2023-08-07,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1


In [9]:
df[df.duplicated(['lead_unique_token','cif','card_no'], keep=False)].sort_values(by='lead_unique_token')

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,dropoff_month,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff
448,QiK-AUfo,6296702,11794976,498796xxxxxx1999,TRUONG VAN THONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-09 13:52:42.000009,2023-08-09,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1617,QiK-AUfo,6296702,11794976,498796xxxxxx1999,TRUONG VAN THONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-09 13:52:42.000009,2023-08-09,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
1582,QiK948pL,6175908,09499159,498796xxxxxx6554,BUI TIEN THANH,VISA EVO SILVER CARD (9023),2023-08-03 18:01:11.000064,2023-08-04,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
1675,QiK948pL,6175908,09499159,498796xxxxxx6554,BUI TIEN THANH,VISA EVO SILVER CARD (9023),2023-08-03 18:01:11.000064,2023-08-04,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1858,QiK96CmB,6193816,11813079,498796xxxxxx5199,NGUYEN HOANG TAM,VISA EVO SILVER CARD (9023),2023-08-01 20:21:56.000016,2023-08-02,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,kRXv1qo-,6270292,03221257,498796xxxxxx0941,LE THI THANH TUYEN,VISA EVO SILVER CARD (9023),2023-08-02 00:36:00.000000,2023-08-09,2023-08,2023-08,2023-08,SMS-MBF WITH OFFER,1,1,1
2215,kRXvkY4w,6234908,02021847,498796xxxxxx1558,DUONG NGOC THU,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 13:48:41.999990,2023-08-07,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
245,kRXvkY4w,6234908,02021847,498796xxxxxx1558,DUONG NGOC THU,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 13:48:41.999990,2023-08-07,2023-07,2023-08,2023-08,EVO_DIGITAL,1,1,1
1833,kRXwFTuc,6302327,10145525,498796xxxxxx4100,NGUYEN VAN PHI,VISA EVO SILVER CARD (9023),2023-08-01 19:24:24.999984,2023-08-07,2023-08,2023-08,2023-08,EVO_DIGITAL,1,1,1


In [10]:
df_dedup = df.loc[:, df.columns != 'dropoff_month'].drop_duplicates()

In [11]:
df_dedup[df_dedup.duplicated(['lead_unique_token'], keep=False)]

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff


In [12]:
df_dedup[df_dedup.duplicated(['cif'], keep=False)]

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff


## add pricing

In [13]:
df_dedup.head()

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff
0,UiK-GuGH,6401560,01580490,498796xxxxxx8945,TRUONG HOANG HUY,VISA CREDIT GOLD EVO CARD (9024),2023-08-11 16:13:39.000057,2023-08-11,2023-08,2023-08,EVO_DIGITAL,1,1,0
1,YRFfBE3y,6326718,04028607,498796xxxxxx3049,NGUYEN MINH THANH,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 11:45:56.999952,2023-08-08,2023-08,2023-08,EVO_DIGITAL,1,1,1
2,QiK97rHg,6220574,02009151,498796xxxxxx2147,NGUYEN THI MY LINH,VISA CREDIT GOLD EVO CARD (9024),2023-08-05 09:00:08.000035,NaT,2023-08,None,SMS-MBF WITH OFFER,1,0,1
3,cRFfBLtq,6330221,11830786,498796xxxxxx9377,NGUYEN ANH HAO,VISA EVO SILVER CARD (9023),2023-08-03 17:47:18.999945,2023-08-07,2023-08,2023-08,EVO_DIGITAL,1,1,0
4,eRFfCkzv,6375837,11888418,498796xxxxxx5729,NGUYEN THI NHUNG,VISA EVO SILVER CARD (9023),2023-08-09 15:57:52.999977,NaT,2023-08,None,EVO_DIGITAL,1,0,1


In [14]:
def label_price (row):
   if row['has_dropoff'] == 1 :
      return 240000
   return 0

In [15]:
df_dedup['price'] = df_dedup.apply (lambda row: label_price(row), axis=1)

In [16]:
df_dedup

,lead_unique_token,appid,cif,card_no,ten_kh,smv_card_category,smv_issued_at,smv_active_at,smv_issue_month,smv_active_month,mkt_channel,has_card_issued,has_card_active,has_dropoff,price
0,UiK-GuGH,6401560,01580490,498796xxxxxx8945,TRUONG HOANG HUY,VISA CREDIT GOLD EVO CARD (9024),2023-08-11 16:13:39.000057,2023-08-11,2023-08,2023-08,EVO_DIGITAL,1,1,0,0
1,YRFfBE3y,6326718,04028607,498796xxxxxx3049,NGUYEN MINH THANH,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 11:45:56.999952,2023-08-08,2023-08,2023-08,EVO_DIGITAL,1,1,1,240000
2,QiK97rHg,6220574,02009151,498796xxxxxx2147,NGUYEN THI MY LINH,VISA CREDIT GOLD EVO CARD (9024),2023-08-05 09:00:08.000035,NaT,2023-08,None,SMS-MBF WITH OFFER,1,0,1,240000
3,cRFfBLtq,6330221,11830786,498796xxxxxx9377,NGUYEN ANH HAO,VISA EVO SILVER CARD (9023),2023-08-03 17:47:18.999945,2023-08-07,2023-08,2023-08,EVO_DIGITAL,1,1,0,0
4,eRFfCkzv,6375837,11888418,498796xxxxxx5729,NGUYEN THI NHUNG,VISA EVO SILVER CARD (9023),2023-08-09 15:57:52.999977,NaT,2023-08,None,EVO_DIGITAL,1,0,1,240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,eRFfD57S,6419418,11926481,498796xxxxxx3518,NGUYEN THI THAO,VISA EVO SILVER CARD (9023),2023-08-13 20:38:29.999961,2023-08-13,2023-08,2023-08,EVO_DIGITAL,1,1,0,0
2451,YRFfBGx0,6327694,11827529,498796xxxxxx9695,NGUYEN HOAI SON,VISA CREDIT GOLD EVO CARD (9024),2023-08-03 13:04:07.000003,2023-08-03,2023-08,2023-08,EVO_DIGITAL,1,1,0,0
2452,cRFfCaW5,6370487,11872135,498796xxxxxx4718,NGUYEN AN KHUONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-08 12:10:18.000019,2023-08-11,2023-08,2023-08,EVO_DIGITAL,1,1,0,0
2453,eRFfCM82,6363622,04316047,498796xxxxxx5941,NGUYEN NGOC DIEM PHUONG,VISA CREDIT GOLD EVO CARD (9024),2023-08-08 13:54:14.000025,2023-08-13,2023-08,2023-08,EVO_DIGITAL,1,1,1,240000


In [17]:
df_dedup.groupby(['smv_issue_month']).agg({'has_card_issued': lambda x: x.sum(),
                                                         'has_card_active': lambda x: x.sum(),
                                                         'has_dropoff': lambda x: x.sum(),
                                                         'price': lambda x: x.sum()}).reset_index()

,smv_issue_month,has_card_issued,has_card_active,has_dropoff,price
0,2023-08,2420,1563,884,212160000


## Export to file

In [18]:
df_to_write = df_dedup.query('has_dropoff == 1')

In [19]:
df_to_write.groupby(['smv_issue_month']).agg({'has_card_issued': lambda x: x.sum(),
                                                         'has_card_active': lambda x: x.sum(),
                                                         'has_dropoff': lambda x: x.sum(),
                                                         'price': lambda x: x.sum()}).reset_index()

,smv_issue_month,has_card_issued,has_card_active,has_dropoff,price
0,2023-08,884,580,884,212160000


In [20]:
file_path_csv = f'{path}{recon_month}/tpb_dropoff_recon_{recon_month}.csv'
file_path_excel = f'{path}{recon_month}/tpb_dropoff_recon_{recon_month}.xlsx'

df_to_write.to_excel(file_path_excel,startcol=0, index=False)
df_to_write.to_csv(file_path_csv, index=False)

## Spreadsheet

In [21]:
summary_data = df_to_write.groupby(['smv_issue_month']).agg({'has_dropoff':'sum', 'price':'sum'}).reset_index()

In [22]:
def create_or_get_folder(recon_month, parent_folder_id):
    # Check if folder exists
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    return folder_id[0]

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key[0])
    return wb

def create_or_update_sheet(wb, new_sheet, input_df):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

In [25]:
parent_folder_id = '1HDX2FKQfb9dRjzqEUOFLL-nZnGj8lebo'

folder_id = create_or_get_folder(recon_month=recon_month, parent_folder_id=parent_folder_id)
TPB_drop_off_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "tpb_dropoff_recon_" + recon_month)

create_or_update_sheet(TPB_drop_off_wb, "Details", df_to_write)
create_or_update_sheet(TPB_drop_off_wb, "Summary", summary_data)

In [24]:
bq_repo.insert_into_table_from_csv_file(
        table_id="prj-ts-p-analytic-8057.da_linhnguyen4.drop_off_recon2",
        csv_file_path=file_path_csv,
        write_disposition="WRITE_APPEND"
    )
